# GOLD on MCX

MCX, the Multi-Commodity eXchange of India, has a number of gold futures contracts listed. This is how you explore them on pluto.

In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(RPostgres)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)
library(reshape2)
library(ggrepel)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

#initialize
commoditiesFnO <- CommodityFuturesAndOptions()
equitiesIndiaNse <- EquitiesIndiaNse() #we'll compare the futures returns with that of GOLDBEES etf's later.

In [ ]:
#list all gold contracts on the MCX

commoditiesFnO$McxEod() %>%
    filter(CONTRACT %like% '%gold%') %>%
    group_by(CONTRACT) %>%
    summarize(ST = min(TIME_STAMP, na.rm=T), ET = max(TIME_STAMP, na.rm=T)) %>%
    arrange(ST) %>%
    print()